In [10]:
# choose location (try to change to your own city!)

city = "Sydney"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

In [12]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

/var/folders/6f/yk_bgn4549g2b7hgr23qy74w0000gn/T/ipykernel_86009/2852339428.py:6: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddg = DDGS()


https://current.com/
https://current.com/product-explainer/
https://current.com/qr/qr-homepage/
https://current.com/text-now-sim-card-terms-and-conditions/
https://support.current.com/hc/en-us/sections/360003247253-Getting-Started
https://support.current.com/hc/en-us/articles/4408250104859-How-do-I-log-in-to-my-Current-Account


In [13]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [14]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

Website: https://current.com/


None


In [15]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://current.com/


Paycheck Advance Disclosures Paycheck Advance is for eligible customers only. Your actual available Paycheck Advance amount will be displayed to you in the mobile app and may change from time to time. Conditions and eligibility may vary and are subject to change at any time, at the sole discretion of Finco Advance LLC, which offers this optional feature. Finco Advance LLC is a financial technology company, not a bank. For more information, please refer to Paycheck Advance Terms and Conditions . Paycheck Advance Disclosures Paycheck Advance is for eligible customers only. Your actual available Paycheck Advance amount will be displayed to you in the mobile app and may change from time to time. Conditions and eligibility may vary and are subject to change at any time, at the sole discretion of Finco Advance LLC, which offers this optional feature. Finco Advance LLC is a financial technology company, not a bank. For more information, please refer to Paycheck

In [16]:
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KE"))

In [ ]:
# test run search
result = client.search("What is in Nvidia's new Blackwell GPU?", include_answer=True)

# print the answer
result["answer"]

In [17]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'Sydney', 'region': 'New South Wales', 'country': 'Australia', 'lat': -33.8833, 'lon': 151.2167, 'tz_id': 'Australia/Sydney', 'localtime_epoch': 1753134447, 'localtime': '2025-07-22 07:47'}, 'current': {'last_updated_epoch': 1753134300, 'last_updated': '2025-07-22 07:45', 'temp_c': 11.2, 'temp_f': 52.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 7.8, 'wind_kph': 12.6, 'wind_degree': 349, 'wind_dir': 'N', 'pressure_mb': 1019.0, 'pressure_in': 30.09, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 94, 'cloud': 75, 'feelslike_c': 9.7, 'feelslike_f': 49.4, 'windchill_c': 12.5, 'windchill_f': 54.4, 'heatindex_c': 13.5, 'heatindex_f': 56.3, 'dewpoint_c': 10.3, 'dewpoint_f': 50.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 13.2, 'gust_kph': 21.3}}


In [18]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)

{
    "location": {
        "name": "Sydney",
        "region": "New South Wales",
        "country": "Australia",
        "lat": -33.8833,
        "lon": 151.2167,
        "tz_id": "Australia/Sydney",
        "localtime_epoch": 1753134447,
        "localtime": "2025-07-22 07:47"
    },
    "current": {
        "last_updated_epoch": 1753134300,
        "last_updated": "2025-07-22 07:45",
        "temp_c": 11.2,
        "temp_f": 52.2,
        "is_day": 1,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
            "code": 1003
        },
        "wind_mph": 7.8,
        "wind_kph": 12.6,
        "wind_degree": 349,
        "wind_dir": "N",
        "pressure_mb": 1019.0,
        "pressure_in": 30.09,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 94,
        "cloud": 75,
        "feelslike_c": 9.7,
        "feelslike_f": 49.4,
        "windchill_c": 12.5,
        "windchill_f": 54.